We initialize Python imports and opens a DuckDB connection that every later cell reuses. We also 

In [ ]:
from pathlib import Path
import duckdb

DB_DIR = Path("../db")
DB_DIR.mkdir(parents=True, exist_ok=True)

con = duckdb.connect(str(DB_DIR / "nflpa.duckdb"))

TEAM_ABBR_COL = "team"
SEASON_COL = "season"
WEEK_COL = "week"

def _existing_cols(table_name: str) -> list[str]:
    df = con.execute(f"DESCRIBE {table_name}").df()
    return df["column_name"].tolist()

def _star_excluding(table_name: str, alias: str, exclude_cols: list[str]) -> str:
    cols = _existing_cols(table_name)
    keep = [c for c in cols if c not in set(exclude_cols)]
    return ",\n  ".join([f'{alias}."{c}"' for c in keep])

def _first_present(cols: list[str], options: list[str]) -> str | None:
    s = set([c.lower() for c in cols])
    for opt in options:
        if opt.lower() in s:
            for c in cols:
                if c.lower() == opt.lower():
                    return c
    return None